<a href="https://colab.research.google.com/github/ovad4221/prevention_of_damage_from_accidents/blob/TecHeReTiC_branch_analys/Analysis_ovad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pprint import pprint
%matplotlib inline

# Алгоритм выявления рекомендаций

In [3]:
viol = pd.read_excel('https://github.com/ovad4221/prevention_of_damage_from_accidents/blob/analisys_main/violations_brief.xlsx?raw=true')
viol.head()

,Unnamed: 0,index,coordL,coordW,place,district,DTPType,date,real_date,month,season,NVehicles,NPatricipants,NInjured,NFatal,additionalFactor,lightConditions,weatherConditions,placeConditions,closeObjects,violations,violationsAdditional,privod,traffic_regime,pokritie,neispravnosti,safety_belt,sublocality,traffic_regime_index,neispravnosti_index,privod_index,RITA,FAIL,violations_brief
0,0,0,37.464817,55.758116,ул Мнёвники Нижн.,Хорошево-мневники,Наезд на препятствие,01.01.2020,2020-01-01,1,Winter,1,1,1,0,Сведения отсутствуют,0.5,Пасмурно,Перегон (нет объектов на месте ДТП),Остановка общественного транспорта,Несоответствие скорости конкретным условиям дв...,Нет нарушений,передний,Режим движения не изменялся,Мокрое,Технические неисправности отсутствуют,Нет,Северо-Западный административный округ,0.0,0,10,125.5,2.5,Скоростной режим
1,1,1,37.388953,55.811338,Московская кольцевая автомобильная дорога (МКА...,Строгино,Наезд на стоящее ТС,01.01.2020,2020-01-01,1,Winter,2,2,1,0,"Наличие на дороге объектов, мешающих движению",1.0,Ясно,Перегон (нет объектов на месте ДТП),Отсутствие в непосредственной близости объекто...,Несоответствие скорости конкретным условиям дв...,Нет нарушений,передний,Движение частично перекрыто,Сухое,Технические неисправности отсутствуют,Нет,Северо-Западный административный округ,165.0,0,10,391.0,3.5,Скоростной режим
2,2,2,37.623153,55.866787,Юрловский пр-д,Отрадное,Наезд на пешехода,01.01.2020,2020-01-01,1,Winter,1,2,1,0,Сужение проезжей части,1.0,Пасмурно,Внутридворовая территория,Административные здания,Несоблюдение очередности проезда,Другие нарушения ПДД водителем,праворукий,Режим движения не изменялся,Сухое,Технические неисправности отсутствуют,Нет,Северо-Восточный административный округ,0.0,0,15,131.0,3.5,Очередь проезда
3,3,3,37.588413,55.660576,ул Болотниковская,Зюзино,Столкновение,01.01.2020,2020-01-01,1,Winter,2,3,1,0,Сведения отсутствуют,0.5,Пасмурно,Регулируемый пешеходный переход,Остановка общественного транспорта,Несоблюдение очередности проезда,Нет нарушений,передний,Движение частично перекрыто,Мокрое,Технические неисправности отсутствуют,Нет,Юго-Западный административный округ,195.0,0,10,421.5,4.5,Очередь проезда
4,4,4,37.636467,55.805098,пр-кт Мира,Алексеевский,Падение чего-либо,01.01.2020,2020-01-01,1,Winter,1,2,1,0,"Участок, контролируемый камерой",0.5,Ясно,Остановка общественного транспорта,Остановка общественного транспорта,Нарушение правил пользования общественным тран...,Нарушение правил перевозки людей,передний,Режим движения не изменялся,Сухое,Технические неисправности отсутствуют,Нет,Северо-Восточный административный округ,0.0,0,10,126.0,3.5,Общ. транспорт


## Динамика

по сезонности (district, season)

In [4]:
set_inst = set([(viol.loc[i, 'district'], viol.loc[i, 'season']) for i in range(len(viol))])
dict_inst = dict()
for i in set_inst:
  dict_inst[i] = []
print(dict_inst)

{('Братеево', 'Spring'): [], ('Северное Медведково', 'Winter'): [], ('Хорошевский', 'Autumn'): [], ('Кузьминки', 'Autumn'): [], ('Щербинка', 'Summer'): [], ('Замоскворечье', 'Summer'): [], ('Бирюлево Западное', 'Winter'): [], ('Печатники', 'Autumn'): [], ('Коньково', 'Summer'): [], ('Кленовское, Поселение', 'Spring'): [], ('Коньково', 'Autumn'): [], ('Алексеевский', 'Spring'): [], ('Печатники', 'Summer'): [], ('Алтуфьевский', 'Spring'): [], ('Строгино', 'Autumn'): [], ('Нагатинский затон', 'Autumn'): [], ('Южнопортовый', 'Summer'): [], ('Крюково', 'Winter'): [], ('Восточный', 'Spring'): [], ('Текстильщики', 'Spring'): [], ('Вороновское, Поселение', 'Autumn'): [], ('Тимирязевский', 'Summer'): [], ('Матушкино', 'Spring'): [], ('Бирюлево Восточное', 'Summer'): [], ('Головинский', 'Spring'): [], ('Восточное Дегунино', 'Summer'): [], ('Северное Медведково', 'Spring'): [], ('Преображенское', 'Summer'): [], ('Матушкино', 'Summer'): [], ('Тверской', 'Winter'): [], ('Ивановское', 'Summer'): [],

In [6]:
def convert(matrix):
  help_dict = {}
  for elem in matrix:
    if elem[0] in help_dict:
      help_dict[elem[0]][0] += elem[1]
      help_dict[elem[0]][1] += 1
    else:
      help_dict[elem[0]] = [elem[1], 1]
  for i in help_dict:
    help_dict[i] = help_dict[i][0] / help_dict[i][1]
  return pd.Series(help_dict)

In [5]:
for i in range(len(viol)):
  dict_inst[(viol.loc[i, 'district'], viol.loc[i, 'season'])].append([viol.loc[i, 'violations_brief'], viol.loc[i, 'RITA']])
print(dict_inst)

{('Братеево', 'Spring'): [['Скоростной режим', 1142.5], ['Очередь проезда', 426.5], ['Игнор светофора', 426.5], ['Выезд на встречку', 226.5]], ('Северное Медведково', 'Winter'): [['Пешеходный переход', 126.0], ['Игнор светофора', 126.0], ['Правила перестроения', 226.5], ['Пешеходный переход', 126.0], ['Пешеходный переход', 126.0], ['Скоростной режим', 226.0], ['Выбор дистанции', 231.5], ['Пешеходный переход', 126.0], ['Выбор дистанции', 226.5], ['Пешеходный переход', 126.0], ['Выбор дистанции', 326.5], ['Пешеходный переход', 126.0], ['Выбор дистанции', 231.0], ['Скоростной режим', 442.0], ['Правила перестроения', 221.5], ['Очередь проезда', 221.0]], ('Хорошевский', 'Autumn'): [['Выбор дистанции', 226.5], ['Выбор дистанции', 226.5], ['Задний ход', 126.0], ['Расположение ТС', 221.0], ['Расположение ТС', 271.0], ['Задний ход', 126.0], ['Очередь проезда', 246.0], ['Выбор дистанции', 337.0], ['Пешеходный переход', 146.0]], ('Кузьминки', 'Autumn'): [['Игнор светофора', 241.0], ['Игнор светоф

In [ ]:
for key in dict_inst:
  dict_inst[key] = convert(dict_inst[key])
print(dict_inst)

In [8]:
viol_dict = {}
for key in dict_inst:
  seri = dict_inst[key]
  for viol in seri.index:
    if viol in viol_dict:
      if key in viol_dict[viol]:
        viol_dict[viol][key][0] += seri[viol]
        viol_dict[viol][key][1] += 1
      else:
        viol_dict[viol][key] = [seri[viol], 1]
    else:
      viol_dict[viol] = {key: [seri[viol], 1]}

for key in viol_dict:
  for key1 in viol_dict[key]:
    viol_dict[key][key1] = viol_dict[key][key1][0] / viol_dict[key][key1][1]

In [ ]:
viol_dict

In [11]:
blowouts = {i: [] for i in viol_dict}

In [49]:
def locate_blowouts(key, dict_value):
  plt.figure(figsize=(10, 8))
  global blowouts
  arr = np.array(list(dict_value.values()))
  plt.boxplot(arr)
  plt.title(f'{key} - boxplot по avg RITA')
  plt.ylabel('Среднее значение RITA')
  plt.savefig(f'{key} - boxplot по avg RITA.png')
  plt.gcf().clear()
  # pprint(sorted(arr))
  Q1 = np.percentile(arr, 25)
  Q3 = np.percentile(arr, 75)
  razm = (Q3 - Q1) * 3
  print(Q1, Q3, razm)
  for i in dict_value:
    if dict_value[i] > Q3 + razm:
      print(f'Выброс при обстоятельствах {i} - {dict_value[i]}')
      blowouts[key].append(f'Выброс при обстоятельствах {i} - {dict_value[i]}')
  
    

In [50]:
for key in viol_dict:
  print(f'{key}:')
  locate_blowouts(key, viol_dict[key])
  print('\n\n\n')

Скоростной режим:
129.84375 328.90625 597.1875
Выброс при обстоятельствах ('Братеево', 'Spring') - 1142.5
Выброс при обстоятельствах ('Даниловский', 'Spring') - 928.0
Выброс при обстоятельствах ('Северное Тушино', 'Spring') - 3891.5
Выброс при обстоятельствах ('Кленовское, Поселение', 'Summer') - 1096.0
Выброс при обстоятельствах ('Чертаново Северное', 'Winter') - 2436.5
Выброс при обстоятельствах ('Кунцево', 'Autumn') - 1007.0




Очередь проезда:
226.0 333.03125 321.09375
Выброс при обстоятельствах ('Сокольники', 'Winter') - 876.0
Выброс при обстоятельствах ('Новофедоровское, Поселение', 'Winter') - 716.0
Выброс при обстоятельствах ('Соколиная Гора', 'Spring') - 913.5
Выброс при обстоятельствах ('Филимонковское, Поселение', 'Autumn') - 726.0
Выброс при обстоятельствах ('Капотня', 'Autumn') - 867.5
Выброс при обстоятельствах ('Ясенево', 'Spring') - 729.0
Выброс при обстоятельствах ('Можайский', 'Spring') - 898.5




Игнор светофора:
224.875 371.95 441.22499999999997
Выброс при обстоят

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

...

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>